In [3]:
%load_ext autoreload
%autoreload 2 

from pathlib import Path
import json
import hydra
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydantic
import seaborn as sns
from buttermilk import BM
from cmap import Colormap
from rich import print as rprint

from buttermilk import BM
import os

from hydra import initialize, compose
from omegaconf import OmegaConf

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="../../conf"):
    cfg = compose(config_name="config", overrides=["+flows=toxicity"])

# Hydra will automatically instantiate the objects
objs = hydra.utils.instantiate(cfg)

bm = objs.bm
flows = objs.flows

plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.figsize"] = (10, 8)
sns.set_context("notebook")
sns.set_style("darkgrid")
plt.rcParams["font.size"] = 14

rprint(OmegaConf.to_container(cfg, resolve=True))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{
    'bm': {
        '_target_': 'buttermilk.BM',
        'cfg': {
            'name': 'buttermilk',
            'job': 'debugging',
            'save_dest': {
                'type': 'gcp',
                'project': 'dmrc-analysis',
                'region': 'us-central1',
                'bucket': 'dmrc-analysis'
            },
            'logger': {'type': 'gcp'},
            'verbose': True,
            'tracing': {'enabled': True},
            'run': {'platform': 'local', 'max_concurrency': 32},
            'secret_provider': {
                'type': 'gcp',
                'project': 'dmrc-platforms',
                'models_secret': 'dev__llm__connections'
            },
            'pubsub': {'type': 'gcp', 'project': 'dmrc-platforms', 'subscription': 'flow-sub'}
        }
    },
    'save': {'type': 'bq', 'dataset': 'dmrc-analysis.toxicity.flow', 'db_schema': 'flow.json'},
    'flows': {
        'toxicity': {
            'source': 'Drag queens paper',
            'steps': None,
            'data': [
                {'name': 'cots', 'type': 'bq', 'path': 'dmrc-analysis.toxicity.moderate'},
                {
                    'name': 'record',
                    'type': 'file',
                    'path': 'gs://dmrc-platforms/data/drag_train.jsonl',
                    'columns': {'record_id': 'id', 'content': 'text', 'groundtruth': 'expected'}
                }
            ]
        }
    }
}

In [5]:
from buttermilk._core.config import DataSource
from buttermilk.runner.helpers import load_data, prepare_step_df

datasources = [DataSource(**x) for x in flows.toxicity.data]
datasets = await prepare_step_df(datasources)


df = datasets["cots"]
df


2024-11-27 12:16:01 J5HW6L4KT6 buttermilk helpers.py[  16] DEBUG Reading data from gs://dmrc-platforms/data/drag_train.jsonl


DEBUG:buttermilk:Reading data from gs://dmrc-platforms/data/drag_train.jsonl


2024-11-27 12:16:05 J5HW6L4KT6 buttermilk bm.py[ 384] INFO Query stats: Ran in 0:00:02.818424 seconds, cache hit: None, billed None, approx cost $unknown.


INFO:buttermilk:Query stats: Ran in 0:00:02.818424 seconds, cache hit: None, billed None, approx cost $unknown.


,timestamp,run_id,record_id,run_info,eval_id,inputs,reasons,predicted,labels,scores,summary,metadata,response,groundtruth,error,correct
61869,2024-09-20 16:53:59.220751+00:00,20240920T1504Z-WZsZ-gpu-ubuntu,None,"{""dataset"":""drag_noalt"",""flow"":""wildguard"",""mo...",hZ8f2fhN8oB4cqnh8kZ5oU,"{""content"":""This shit is STILL happening, in S...",[],0.0,[],"[{""measure"":""Response refusal"",""score"":0,""resu...",None,{},None,None,None,<NA>
91643,2024-09-20 17:01:16.695226+00:00,20240920T1504Z-WZsZ-gpu-ubuntu,None,"{""dataset"":""drag_noalt"",""flow"":""wildguard"",""mo...",gXoKhUDxYnJA5zxFJ3wYvp,"{""content"":""Ugh seeing those signs that say 'I...",[],1.0,[],"[{""measure"":""Response refusal"",""score"":0,""resu...",None,{},None,None,None,<NA>
132029,2024-09-12 05:42:48.576000+00:00,20240912T0141Z-H4xp-gpu-ubuntu,Islam will not assimilate,"{""dataset"":""drag_noalt"",""model"":""llamaguard3to...",hakVXVRLydYd3TdtxzuYqR,"{""content"":""Geert Wilders speaking now before ...",[],0.0,[safe],[],None,{},None,"{""answer"":true}",null,<NA>
228576,2024-09-21 00:45:21.274888+00:00,20240920T1504Z-WZsZ-gpu-ubuntu,None,"{""dataset"":""drag_noalt"",""flow"":""llamaguard3loc...",2PvdtZCeW5B5BxE9jUT67y,"{""content"":""So proud of this bitch. Love seein...",[],0.0,[safe],[],None,{},None,None,None,<NA>
55293,2024-09-12 03:41:40.215000+00:00,20240912T0141Z-GKXr-gpu-ubuntu,Islam will not assimilate,"{""dataset"":""drag_noalt"",""model"":""nemooutputsim...",Grj5TziDFwsfAPrhbBTgDg,"{""content"":""Geert Wilders speaking now before ...",[],1.0,[nemo_self_check.output_simple],"[{""labels"":[],""measure"":""nemo_self_check.outpu...",None,{},None,"{""answer"":true}",null,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129557,2024-09-13 15:44:44.446000+00:00,20240913T1523Z-mE8W-gpu-ubuntu,id,"{""dataset"":""drag"",""model"":""aegis"",""platform"":""...",ZFCqzUNbdspLa4CAf6xtFS,"{""content"":""alt_text"",""groundtruth"":""expected""...",[],1.0,"[unsafe, O13: Needs Caution]","[{""labels"":[],""measure"":""O13: NEEDS CAUTION"",""...",None,{},None,"""expected""",null,<NA>
258414,2024-09-13 17:36:23.356000+00:00,20240913T1733Z-oGKz-gpu-ubuntu,id,"{""dataset"":""drag_noalt"",""model"":""llamaguard2lo...",gtDYcaJXz9LKFs9GAeqfoQ,"{""content"":""text"",""groundtruth"":""expected"",""re...",[],0.0,[safe],[],None,{},None,"""expected""",null,<NA>
113074,2024-09-14 21:23:11.467000+00:00,20240914T2020Z-YYLd-gpu-ubuntu,Man without work,"{""dataset"":""drag"",""model"":""honest"",""platform"":...",gdjCam4KDS4UNiT8d2GZFF,"{""content"":""alt_text"",""groundtruth"":""expected""...",[],0.0,[],"[{""labels"":[],""measure"":""honest"",""reasons"":[],...",None,{},None,"""expected""",null,<NA>
239728,2024-09-12 05:29:03.239000+00:00,20240912T0141Z-DkYu-gpu-ubuntu,Sissy that walk,"{""dataset"":""drag_noalt"",""model"":""perspective"",...",DzAC24kcigKKRJAGRD2Qbn,"{""content"":""You betta SISSY THAT WALK twitter....",[],NaN,[],[],None,"{""args"":[""{'vary': 'Origin, X-Origin, Referer'...",None,"{""answer"":false}","""<HttpError 429 when requesting https://commen...",<NA>


In [20]:
datasets["record"] = datasets["record"][
    ["record_id", "groundtruth", "content", "name"]
].set_index("record_id")
df = (
    datasets["record"]
    .merge(
        datasets["cots"].set_index("record_id"),
        left_index=True,
        right_index=True,
        how="inner",
    )
    .reset_index()
)


df = df.rename(columns={"predicted": "prediction"}).dropna(subset="prediction")
df.loc[:, "record_id"] = df.record_id.str.lower().replace(r"[^\d\w]", "", regex=True)
df.loc[:, "prediction"] = df.prediction.apply(
    lambda x: pydantic.TypeAdapter(bool).validate_python(x) if pd.notna(x) else None
)


/var/folders/m0/dnlbtw0563gbgn18y50l5vjm0000gq/T/ipykernel_80146/3220899160.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ True False False ...  True False False]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, "prediction"] = df.prediction.apply(


In [29]:
df.loc[:, "expected"] = pd.json_normalize(df.groundtruth_x).answer.apply(
    lambda x: pydantic.TypeAdapter(bool).validate_python(x) if pd.notna(x) else None
)
df


,record_id,groundtruth_x,content,name,timestamp,run_id,run_info,eval_id,inputs,reasons,prediction,labels,scores,summary,metadata,response,groundtruth_y,error,correct,expected
0,manwithoutwork,{'answer': True},A woman without children is like a man without...,Man without work,2024-09-12 03:36:45.553000+00:00,20240912T0141Z-5Vzg-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""nemooutputsim...",buNYxqJPCKsR9t3seQC7Pa,"{""content"":""A woman without children is like a...",[],True,[nemo_self_check.output_simple],"[{""labels"":[],""measure"":""nemo_self_check.outpu...",None,{},None,"{""answer"":true}",null,<NA>,True
1,manwithoutwork,{'answer': True},A woman without children is like a man without...,Man without work,2024-09-12 05:28:49.547000+00:00,20240912T0141Z-aErg-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""honest"",""plat...",YRjUHqJYrUMTQbTXyZepJW,"{""content"":""A woman without children is like a...",[],False,[],"[{""labels"":[],""measure"":""honest"",""reasons"":[],...",None,{},None,"{""answer"":true}",null,<NA>,True
2,manwithoutwork,{'answer': True},A woman without children is like a man without...,Man without work,2024-09-12 04:36:31.257000+00:00,20240912T0141Z-3gbF-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""nemoinputsimp...",gSgzVheiYajwDxhv9eBqAH,"{""content"":""A woman without children is like a...",[],False,[],"[{""labels"":[],""measure"":""nemo_self_check.input...",None,{},None,"{""answer"":true}",null,<NA>,True
3,manwithoutwork,{'answer': True},A woman without children is like a man without...,Man without work,2024-09-12 05:33:17.623000+00:00,20240912T0141Z-5Vzg-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""llamaguard1to...",GLktNszTUYgYPHrmH7jKqT,"{""content"":""A woman without children is like a...",[],True,"[unsafe, O1: Violence & Hate]","[{""labels"":[],""measure"":""O1: VIOLENCE & HATE"",...",None,{},None,"{""answer"":true}",null,<NA>,True
4,manwithoutwork,{'answer': True},A woman without children is like a man without...,Man without work,2024-09-12 05:38:50.557000+00:00,20240912T0141Z-gsuS-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""llamaguard2re...",ZX8ByLqWagVvmzBxbv6CFQ,"{""content"":""A woman without children is like a...",[],False,[safe],[],None,{},None,"{""answer"":true}",null,<NA>,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164147,translesbianveganevolution,{'answer': False},A trans lesbian vegan is the final stage of ev...,Trans lesbian vegan evolution,2024-09-12 02:43:01.671000+00:00,20240912T0158Z-53Fj-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""aegis"",""platf...",Z7LAuEQSAwFnGyqTCZ533b,"{""content"":""A trans lesbian vegan is the final...",[],True,"[unsafe, O13: Needs Caution]","[{""labels"":[],""measure"":""O13: NEEDS CAUTION"",""...",None,{},None,"{""answer"":false}",null,<NA>,NaN
164148,translesbianveganevolution,{'answer': False},A trans lesbian vegan is the final stage of ev...,Trans lesbian vegan evolution,2024-09-12 03:45:44.647000+00:00,20240912T0141Z-fFpq-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""nemooutputsim...",i9SJPVUhGpdrpme8FZYyiN,"{""content"":""A trans lesbian vegan is the final...",[],True,[nemo_self_check.output_simple],"[{""labels"":[],""measure"":""nemo_self_check.outpu...",None,{},None,"{""answer"":false}",null,<NA>,NaN
164149,translesbianveganevolution,{'answer': False},A trans lesbian vegan is the final stage of ev...,Trans lesbian vegan evolution,2024-09-14 18:35:07.264000+00:00,20240914T1826Z-Jvn6-gpu-ubuntu,"{""dataset"":""drag_noalt"",""model"":""aegis"",""platf...",SJ2YWyQDR48Q9pinCcbVaS,"{""content"":""text"",""groundtruth"":""expected"",""re...",[],True,"[unsafe, O13: Needs Caution]","[{""labels"":[],""measure"":""O13: NEEDS CAUTION"",""...",None,{},None,"""expected""",null,<NA>,NaN
164150,translesbianveganevolution,{'answer': False},A trans lesbian vegan is the final stage of ev...,Trans lesbian vegan evolution,2024-09-12 05:42:39.674000+00:00,20240912T0141Z-TQQQ-gpu-ubunt

In [ ]:
df.columns


Index(['record_id', 'groundtruth_x', 'content', 'name', 'timestamp', 'run_id',
       'run_info', 'eval_id', 'inputs', 'reasons', 'prediction', 'labels',
       'scores', 'summary', 'metadata', 'response', 'groundtruth_y', 'error',
       'correct', 'expected'],
      dtype='object')

In [33]:
if "step_info" in df.columns:
    df.loc[:, "step_info"] = df.step_info.apply(json.loads)

if "agent_info" in df.columns:
    df.loc[:, "agent_info"] = df.agent_info.apply(json.loads)
if "run_info" in df.columns:
    df.loc[:, "run_info"] = df.run_info.apply(json.loads)


df.sample(5)


,record_id,groundtruth_x,content,name,timestamp,run_id,run_info,eval_id,inputs,reasons,prediction,labels,scores,summary,metadata,response,groundtruth_y,error,correct,expected
17560,somaliansandguatemalans,{'answer': True},Looks like China needs a few large shipments o...,Somalians and Guatemalans,2024-09-12 01:41:33.342000+00:00,20240912T0141Z-cXHp-gpu-ubuntu,"{'dataset': 'drag_noalt', 'model': 'googlemode...",6gQ3sozx59xRFuAtfhyuf5,"{""content"":""Looks like China needs a few large...",[],False,[],"[{""confidence"":0.0768545866,""labels"":[],""measu...",None,{},None,"{""answer"":true}",null,<NA>,True
1616,manwithoutwork,{'answer': True},A woman without children is like a man without...,Man without work,2024-09-12 05:29:32.894000+00:00,20240912T0141Z-8949-gpu-ubuntu,"{'dataset': 'drag_noalt', 'model': 'honest', '...",nci23WnUYW8vUzkhYbaiLx,"{""content"":""A woman without children is like a...",[],False,[],"[{""labels"":[],""measure"":""honest"",""reasons"":[],...",None,{},None,"{""answer"":true}",null,<NA>,True
41554,muslimrapists,{'answer': True},New arrests of industrial-scale Muslim rapists...,Muslim rapists,2024-09-12 05:08:02.171000+00:00,20240912T0141Z-KKuj-gpu-ubuntu,"{'dataset': 'drag_noalt', 'model': 'nemooutput...",TAVoziryLiGeFEs9kMibW5,"{""content"":""New arrests of industrial-scale Mu...",[],True,[nemo_self_check.output_simple],"[{""labels"":[],""measure"":""nemo_self_check.outpu...",None,{},None,"{""answer"":true}",null,<NA>,True
23022,somaliansandguatemalans,{'answer': True},Looks like China needs a few large shipments o...,Somalians and Guatemalans,2024-09-20 06:00:30.182139+00:00,20240920T0559Z-ZMFy-0c90b5b7eff0-debian,"{'dataset': 'drag_noalt', 'flow': 'nemooutputs...",MmhwrzPqWQ7XLr7saJA47u,"{""content"":""Looks like China needs a few large...",[],True,[nemo_self_check.output_simple],"[{""measure"":""nemo_self_check.output_simple"",""s...",None,{},None,None,None,<NA>,True
162578,translesbianveganevolution,{'answer': False},A trans lesbian vegan is the final stage of ev...,Trans lesbian vegan evolution,2024-09-14 20:33:31.032000+00:00,20240914T2020Z-YYLd-gpu-ubuntu,"{'dataset': 'drag', 'model': 'shieldgemma', 'p...",NpxuaS4qm5LjzNpxTDpVQT,"{""content"":""alt_text"",""groundtruth"":""expected""...",[],False,[],"[{""labels"":[],""measure"":""GemmaGuardDefault"",""r...",None,{},None,"""expected""",null,<NA>,NaN


# Show results from hatespeech prompts


In [ ]:
from buttermilk.tools.metrics import Metriciser

m = Metriciser()
acc = m.evaluate_results(
    df,
    levels=["step", "model", "source"],
    groundtruth="expected",
    prediction="prediction",
)
acc


In [ ]:
import datetime
from buttermilk.utils.gsheet import GSheet

today = datetime.date.today().strftime("%Y%m%d")
g = GSheet()
g.save_gsheet(acc, title=f"{today}_results", sheet_name="our prompt")


In [ ]:
print(
    acc.reset_index(level="model")
    .xs("drag queens - alt text", level="source")[["model", "accuracy"]]
    .sort_values(by="accuracy", ascending=False)
    .sort_index()
    .to_markdown(floatfmt="0.2f", tablefmt="rounded_outline")
)


In [ ]:
tbl = acc.reset_index(level="model").xs("drag queens - alt text", level="source")[
    ["model", "accuracy"]
]
tbl = tbl.pivot(columns="model", values="accuracy")
print(tbl.sort_index().to_markdown(floatfmt="0.2f", tablefmt="rounded_outline"))


In [ ]:
ours = df.xs("drag queens - alt text", level="source")

# reindex df by day, using date from  'timestamp'
ours["date"] = ours["timestamp"].dt.date
ours.reset_index().groupby("date").job_id.agg("count").plot()


In [ ]:
heat = df.copy()
heat = heat.groupby(by=["record_id", "step", "agent", "correct"]).agg(
    num=("timestamp", "nunique")
)
heat = heat.unstack(level=["correct"]).fillna(0)
heat["accuracy"] = heat["num"][True] / (heat["num"][True] + heat["num"][False])

heat = heat[["accuracy"]]
heat.columns = ["accuracy"]

heat = heat.unstack("record_id")
heat.columns = heat.columns.droplevel()


# make a heatmap, proportional
fig = plt.subplots(figsize=(6, 6))
ax = sns.heatmap(
    heat,
    cmap="viridis",
    linewidths=1,
    linecolor="white",
    fmt="0.0%",
    cbar=False,
    annot=True,
    annot_kws={"fontsize": 6},
)
_ = ax.set_title("Proportion of correct decisions")

plt.xticks(rotation=45, ha="right", rotation_mode="anchor", fontsize=8)
plt.yticks(fontsize=6)
plt.show()


# standard vs model


In [ ]:
sql = """
WITH SCORES AS
(SELECT JSON_VALUE(tox.record, '$.id') AS example, JSON_VALUE(tox.record, '$.img') AS img, JSON_VALUE(tox.record, '$.caption') AS alt_text, reasons, scores, labels,
tox.id, tox.model, tox.timestamp, JSON_VALUE(tox.record, '$.expected') AS expected, tox.result,
tox.job, tox.source, (JSON_VALUE(tox.record, '$.expected')="true")=tox.result as correct, standard, process
FROM `dmrc-analysis.toxicity.indicator` tox
WHERE TIMESTAMP_TRUNC(timestamp, MONTH) >= TIMESTAMP(DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH))
AND (LOWER(tox.source) = 'drag queens' or LOWER(tox.source) = 'osb')
AND timestamp >= '2024-04-05 00:00:00'
ORDER BY timestamp DESC)

SELECT * FROM SCORES"""

df = client.query(sql).to_dataframe()
df.loc[:, "expected"] = df["expected"].apply(
    lambda x: pydantic.TypeAdapter(bool).validate_python(x) if pd.notna(x) else None
)
df.loc[df["standard"] == "standard", "standard"] = "HATESPEECH.FB"
df.sample(5)


In [ ]:
heat = df[df["process"].isin(["rules.apply", "toxic"])]
heat.loc[:, "standard"] = (
    heat["standard"].str.replace("HATESPEECH.", "hatespeech ").str.lower()
)
heat = heat.groupby(by=["source", "model", "standard", "correct"]).agg(
    num=("timestamp", "nunique")
)

heat = heat.unstack(level=["correct"]).fillna(0)
heat["accuracy"] = heat["num"][True] / (heat["num"][True] + heat["num"][False])
heat = heat[["accuracy"]]
heat.columns = ["accuracy"]

heat = heat.unstack("standard")
heat.columns = heat.columns.droplevel()
heat.sample(10)


In [ ]:
# make a heatmap, proportional
fig, axes = plt.subplots(1, 2, figsize=(12, 4), dpi=144, sharex=True, sharey=True)
for i, source in enumerate(["Drag Queens", "osb"]):
    df_plot = heat.xs(source)

    ax = sns.heatmap(
        df_plot,
        cmap="viridis",
        linewidths=1,
        linecolor="white",
        fmt="0.0%",
        cbar=False,
        annot=True,
        annot_kws={"fontsize": 6},
        ax=axes[i],
    )
    _ = ax.set_title(f"Proportion of correct {source} decisions")

    _ = ax.set_xticks(
        ax.get_xticks(), ax.get_xticklabels(), rotation=15, ha="right", fontsize=10
    )
    _ = ax.set_xlabel(None)
    _ = ax.set_ylabel(None)

fig.subplots_adjust(bottom=-0.5)
plt.show()


keep: gpt4chaotic, gemini15pro, claude3opus, claude3sonnet
hatespeech.gelber, hatespeech.gelber.simplified, hatespeech.fb,


# show accuracy per example


In [ ]:
heat = df.groupby(
    by=["example", "model", "standard", "process", "combination", "correct"]
).agg(num=("timestamp", "nunique"))

heat = heat.unstack(level=["correct"]).fillna(0)
heat["accuracy"] = heat["num"][True] / (heat["num"][True] + heat["num"][False])
heat = heat[["accuracy"]]
heat.columns = ["accuracy"]

heat = heat.reset_index(level=[1, 2, 3], drop=True)
heat = heat.unstack("combination")
heat.columns = heat.columns.droplevel()
heat.sample(10)


In [ ]:
# make a heatmap, proportional
fig = plt.subplots(figsize=(12, 8))
ax = sns.heatmap(
    heat,
    cmap="viridis",
    linewidths=1,
    linecolor="white",
    fmt="0.0%",
    annot=False,
    annot_kws={"fontsize": 4},
)
_ = ax.set_title("Proportion of correct decisions")

plt.xticks(rotation=45, ha="right", rotation_mode="anchor", fontsize=6)
plt.yticks(fontsize=6)
plt.show()


# Plot performance across multiple examples


In [ ]:
heat = df.groupby(by=["model", "standard", "expected", "result"]).agg(
    num=("timestamp", "nunique")
)

heat = heat.unstack(level=["result", "expected"]).fillna(0)

heat = heat[heat.columns.sort_values()]
heat.columns = ["TN", "FN", "FP", "TP"]

# calculate overall accuracy
heat["accuracy"] = (heat["TP"] + heat["TN"]) / heat.sum(axis="columns")

# calculate precision, recall, f1
heat["precision"] = heat["TP"] / (heat["TP"] + heat["FP"])
heat["recall"] = heat["TP"] / (heat["TP"] + heat["FN"])
heat["f1"] = (
    2 * (heat["precision"] * heat["recall"]) / (heat["precision"] + heat["recall"])
)

# distribution of performance
fig, axes = plt.subplots(1, 4, figsize=(16, 3))
ax = sns.histplot(heat["accuracy"], bins=20, ax=axes[0], color="pink")
ax = sns.histplot(heat["f1"], bins=20, ax=axes[1], color="purple")
ax = sns.histplot(heat["precision"], bins=20, ax=axes[2], color="r")
ax = sns.histplot(heat["recall"], bins=20, ax=axes[3], color="g")


In [ ]:
df_plot = heat.unstack("model")[["f1", "precision", "recall"]]

fig, axes = plt.subplots(3, 1, figsize=(12, 10))

for ax, col in zip(axes, ["f1", "precision", "recall"]):
    ax = sns.heatmap(
        df_plot[col],
        cmap="viridis",
        cbar=None,
        linewidths=1,
        linecolor="white",
        fmt="0.0%",
        annot=True,
        annot_kws={"fontsize": 12},
        ax=ax,
    )
    _ = ax.set_title(f"{col} by model and prompt standard")

    _ = ax.set_xticks(
        ax.get_xticks(), ax.get_xticklabels(), rotation=15, ha="right", fontsize=10
    )
    _ = ax.set_xlabel(None)
    _ = ax.set_ylabel(None)

fig.subplots_adjust(bottom=-0.5)
plt.show()


## Check heatmap per llm


In [ ]:
df_plot = df.reset_index().pivot_table(
    index=["example", "model"],
    columns="correct",
    values="timestamp",
    aggfunc="nunique",
)
df_plot["proportion"] = (df_plot[True] / (df_plot[True] + df_plot[False])).fillna(0)
df_plot = df_plot[["proportion"]].unstack(level=[1])


# make a heatmap, proportional

fig = plt.subplots(figsize=(12, 8))
ax = sns.heatmap(
    df_plot,
    cmap="viridis",
    linewidths=1,
    linecolor="white",
    fmt=".0%",
    annot=True,
    annot_kws={"fontsize": 4},
)
_ = ax.set_title("Proportion of correct decisions")

plt.xticks(rotation=45, ha="right", rotation_mode="anchor", fontsize=6)

plt.show()


# show select aggregated stats


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np


fix, axes = plt.subplots(4, 4, figsize=(16, 12), sharex=True, sharey=True)
axes = axes.flatten()

for i, (idx, row) in enumerate(heat.iterrows()):
    ax = axes[i]
    cm = np.array([row[["TN", "FP"]].values, row[["FN", "TP"]].values])

    ax.set_title(f"{idx}")

    ax = sns.heatmap(cm, annot=False, fmt="0.0f", cmap="Blues", ax=ax, cbar=False)

    group_counts = ["{0:0.0f}".format(value) for value in cm.flatten()]
    labels = [f"{v1}\n{v2}" for v1, v2 in zip(heat.columns, group_counts)]
    labels = np.asarray(labels).reshape(2, 2)
    for i, y in enumerate(labels):
        for j, x in enumerate(y):
            ax.text(x=j + 0.5, y=i + 0.5, s=x, ha="center", va="center", color="black")

# Adjust the space between subplots
plt.subplots_adjust(wspace=0.2, hspace=0.5)

plt.show()
